# Instrument1

## Reference

* https://www.tensorflow.org/lite/microcontrollers/get_started
* https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/hello_world/train/train_hello_world_model.ipynb

## Install Python packages

In [ ]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install matplotlib

## Upload data

## Train neural network

## Convert the trained model to TensorFlow Lite